In [ ]:
import os
from PIL import Image
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.applications import InceptionResNetV2 #299
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D ,Dropout
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.optimizers import Adam


def load(path,target_size,arg = 'yes'):
  ret_data = []
  ret_label = []
  ret_name = []
  count = 0

  for i in path: #path is a list of folder path
    file_list = os.listdir(i)
    # Filter out non-image files (if any)
    img_extensions = ['.jpg', '.jpeg', '.png', '.bmp']
    img_files = [f for f in file_list if os.path.splitext(f)[1].lower() in img_extensions]
    for f in img_files: #f is image name is the folder
        img_path = os.path.join(i, f)
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = img.resize(target_size)
        
        # Original image
        img_array = np.array(img)
        label = np.zeros(120)
        label[count] = 1 #Create one-hot encoding label
        ret_data.append(img_array)
        ret_label.append(label)
        ret_name.append(f)
        
        
    count += 1
  

        
  ret_data , ret_label , ret_name = np.array(ret_data),np.array(ret_label),np.array(ret_name)
  return ret_data,ret_label,ret_name


if __name__ == "__main__":
    #load .mat file
    file_name  = "C:/Users/88696/Desktop/HW3 data/stanford_dogs/stanfor_dogs"
    
    base_model = InceptionResNetV2(include_top= False, input_shape= (224, 224, 3), weights= 'imagenet')

    
    for layer in base_model.layers[0:len(base_model.layers)-20]:  
       layer.trainable = False
        
    model = Sequential()
    # Add your custom top layer
    model.add(base_model)
    model.add(GlobalAveragePooling2D())
    model.add(Dropout(0.1))
    model.add(Dense(1024,activation='relu'))
    model.add(Dense(512,activation='relu'))
    model.add(Dense(120,activation='softmax'))
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1)
    
    # Create the full model
    model.summary()
    
    # Compile the model
    model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
    # Create an instance of ImageDataGenerator
    
    train_datagen  = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        rescale=1./255,
        validation_split=0.15
    )
    
    test_datagen = ImageDataGenerator(
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        fill_mode='nearest',
        rescale=1./255
    )

    
    # load and iterate training dataset
    train_it = train_datagen.flow_from_directory('C:/Users/88696/Desktop/HW3 data/stanford_dogs/stanfor_dogs/train', 
                                                 class_mode='categorical', 
                                                 batch_size=64, 
                                                 target_size=(224, 224),
                                                 subset='training'
                                                 )
    
    val_it = train_datagen.flow_from_directory('C:/Users/88696/Desktop/HW3 data/stanford_dogs/stanfor_dogs/train', 
                                                 class_mode='categorical', 
                                                 batch_size=64, 
                                                 target_size=(224, 224),
                                                 subset='validation'
                                                 )
    
    
    callbacks = [early_stopping, reduce_lr]

    history = model.fit(train_it, validation_data=val_it, epochs=50, steps_per_epoch=len(train_it), 
           callbacks=callbacks)
    
    
    
    
    # Plot the training and validation accuracy
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['acc'])
    plt.plot(history.history['val_acc'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    
    # Plot the training and validation loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(['Train', 'Validation'], loc='upper left')
    plt.tight_layout()
    plt.show()    
        
    #predicting test data
    test_path = [file_name+"/test"]
    test_data , test_label , test_name = load(test_path,(224,224))
    result = model.predict(test_data)
    predicted_labels = np.argmax(result, axis=1)
    #store in xlsx file
    df = pd.DataFrame({
    'Name': test_name,
    'Label': predicted_labels
    })
    
    df.to_excel('C:/Users/88696/Desktop/HW3 data/result.xlsx', index=False)